# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Message', engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def tokenize(text):
    """
    
    Parameters:
    ----------
    text : str
        The input text to be processed.
    
    Returns:
    -------
    tokens : list
        A list of processed words (tokens) that have been normalized, tokenized, cleaned of stopwords, and lemmatized.
    
    """

    # Normalize text: convert to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words("english")]

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Build a machine learning pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),  # Convert text to a matrix of token counts
    ('tfidf', TfidfTransformer()),                 # Apply TF-IDF transformation
    ('clf', MultiOutputClassifier(RandomForestClassifier()))  # Apply classifier for multi-label outputs
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
# splitting the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# Train the model
pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# Make predictions
Y_pred = pipeline.predict(X_test)

# Print classification report for each category
for i, col in enumerate(Y_test.columns):
    print(f'Category: {col}\n', classification_report(Y_test[col], Y_pred[:, i]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define parameters for grid search
parameters = {
    'vect__max_df': [0.75, 1.0],  # Maximum document frequency
    'vect__ngram_range': [(1, 1), (1, 2)],  # Unigrams or bigrams
    'tfidf__use_idf': [True, False],  # Whether to use inverse document frequency
    'clf__estimator__n_estimators': [50, 100],  # Number of trees in RandomForest
    'clf__estimator__min_samples_split': [2, 4]  # Minimum samples required to split an internal node
}

# Set up grid search with cross-validation
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, n_jobs=-1)

# Fit the model using grid search
cv.fit(X_train, Y_train)

# Display the best parameters
print("Best Parameters:", cv.best_params_)

# Make predictions with the best estimator
Y_pred = cv.predict(X_test)

# Print classification report for each category using the best model
for i, col in enumerate(Y_test.columns):
    print(f'Category: {col}\n', classification_report(Y_test[col], Y_pred[:, i]))


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Make predictions using the best model from grid search
Y_pred = cv.predict(X_test)

# Initialize lists to store precision, recall, and f1-score for each category
precision_list = []
recall_list = []
f1_list = []

# Loop through each category and print the classification report
for i, col in enumerate(Y_test.columns):
    print(f'Category: {col}')
    print(classification_report(Y_test[col], Y_pred[:, i]))
    
    # Collect precision, recall, and f1-score for analysis
    report = classification_report(Y_test[col], Y_pred[:, i], output_dict=True)
    precision_list.append(report["weighted avg"]["precision"])
    recall_list.append(report["weighted avg"]["recall"])
    f1_list.append(report["weighted avg"]["f1-score"])

# Calculate average accuracy, precision, recall, and f1-score across all categories
avg_precision = sum(precision_list) / len(precision_list)
avg_recall = sum(recall_list) / len(recall_list)
avg_f1 = sum(f1_list) / len(f1_list)
overall_accuracy = (Y_pred == Y_test.values).mean().mean()

print(f"\nOverall Model Performance:")
print(f"Accuracy: {overall_accuracy:.4f}")
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1-Score: {avg_f1:.4f}")


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

# Replace RandomForestClassifier with AdaBoostClassifier
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

# Add text length as a feature
df['text_length'] = df['message'].apply(len)

# Update X to include this feature
X = pd.concat([X, df['text_length']], axis=1)

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

def add_pos_tags(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    return [tag for word, tag in pos_tags]

df['pos_tags'] = df['message'].apply(add_pos_tags)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion

# Define a feature union for combining text features and text length
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('text_length', FunctionTransformer(lambda x: pd.DataFrame(x['message'].apply(len)), validate=False))
    ])),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

# Fit the pipeline and make predictions as before
pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_test)

### 9. Export your model as a pickle file

In [ ]:
import pickle

# Export the model to a pickle file
with open('classifier_model.pkl', 'wb') as file:
    pickle.dump(cv.best_estimator_, file)


### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.